In [1]:
import torch
from torch import nn
from mmcv.cnn.bricks.conv_module import ConvModule

c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [23]:
class FPN(nn.Module):
    
    def __init__(self,
                in_channels,
                channels,
                conf_cfg=None,
                norm_cfg=None,
                act_cfg=None,
                ):
        super().__init__()
        self.lateral_convs=nn.ModuleList()
        self.fpn_conv =nn.ModuleList()
        self.in_channels=in_channels
        for i in range(len(in_channels)):
            self.lateral_convs.append(
                ConvModule(in_channels[i],
                           channels,1,
                           conv_cfg=conf_cfg,
                           norm_cfg=norm_cfg,
                           act_cfg=None))
        for i in range(len(in_channels)):
            self.fpn_conv.append(
                ConvModule(channels,
                           channels,3,1,1,
                           conv_cfg=conf_cfg,
                           norm_cfg=norm_cfg,
                           act_cfg=None))
    def forward(self,x):
        out = x
        for i in range(len(self.in_channels)):
            out[i] = self.lateral_convs[i](out[i])
        for j in range(len(self.in_channels)-2,-1,-1):
            pre_size = out[j].shape[2:]
            out[j]=out[j]+ F.interpolate(out[j+1],size=pre_size)
        for i in range(len(self.in_channels)):
            out[i] = self.fpn_conv[i](out[i])
        
        return out

In [24]:
fpn = FPN([2, 3, 5, 7],11)

In [25]:
x = [torch.randn(1,64,112,112),torch.randn(1,128,56,56),torch.randn(1,256,28,28),torch.randn(1,512,14,14)]
fpn.eval()
torch.onnx.export(fpn,inputs,'fpn.onnx',11)